In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml
from torch_geometric.datasets import Planetoid
from tqdm import tqdm

import wandb
from GCN import GCN
from utils import build_adj_mat

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
    config = config["GCN"]

wandb.init(project="gnn-from-scratch", config=config)

node_dim = config["node_dim"]
hidden_dim = config["hidden_dim"]
batch_size = config["batch_size"]
lr = config["lr"]
n_classes = config["n_classes"]
n_epochs = config["n_epochs"]
n_train = config["n_train"]
n_val = config["n_val"]
n_test = config["n_test"]
dropout = config["dropout"]
weight_decay = config["weight_decay"]

# dataset = Planetoid(
#     "./data/", "Cora", num_train_per_class=n_train, num_val=n_val, num_test=n_test
# )
dataset = Planetoid(
    "./data/", "PubMed", num_train_per_class=n_train, num_val=n_val, num_test=n_test
)


In [12]:
dataset[0].y.max()

tensor(2)